In [20]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft
import warnings
import autofeat 
from tpot import TPOTClassifier
import json 
import time
from sklearn import preprocessing 

warnings.filterwarnings('ignore')

In [36]:
def run_as(X, y, target_ft, time_budget=30, include_preprocessors = None):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
        os.remove('/tmp/autosklearn_regression_example_out')
    except:
        pass
    #X = df.drop(columns=target_ft)
    #y = df[target_ft]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=time_budget,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
        include_preprocessors = include_preprocessors
    )
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    
    acc = sklearn.metrics.accuracy_score(y_test, y_hat)
    f1_s = sklearn.metrics.f1_score(y_test, y_hat, average='macro')
    metrs = []
    metrs.append("Accuracy score - " + str(acc))
    metrs.append("F1 score - " + str(f1_s))
    
    res = ["","","","",f1_s,acc,"",automl.show_models()]
    
    
    return str(metrs),res

def run_tpot(X,y, target_ft,time_budget=30, include_preprocessors=None ):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    pipeline_optimizer = TPOTClassifier(max_time_mins = time_budget/60, generations=None)
    pipeline_optimizer.fit(X_train, y_train)
    y_hat = pipeline_optimizer.predict(X_test)
    metrs = []
    metrs.append("Accuracy score - " + str(sklearn.metrics.accuracy_score(y_test, y_hat)))
    metrs.append("F1 score - " + str(sklearn.metrics.f1_score(y_test, y_hat, average='macro')))
    return str(metrs)

    
def gen_feats_featools(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data', verbose = 1)
                                      #agg_primitives=["mean", "max", "min", "std", "skew"],
                                     # trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

def gen_feats_autofeat(X,y):
    fsel = autofeat.FeatureSelector(verbose=1)
    X = fsel.fit_transform(X,y)
    return X
    
    
def run_test(df_path,target_ft, mode = 0, time_budget=30):
    results = []
    df = pd.read_csv(df_path)
    df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').apply(preprocessing.LabelEncoder().fit_transform)
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    res_df = []
    autofeat_time = 0
    
    if mode == 0 or mode == 1:
        start = time.monotonic()
        X_new = gen_feats_autofeat(X,y)
        end = time.monotonic()
        autofeat_time = int(end-start)
        print("Autofeat_time: ",autofeat_time)
        rs = run_as(X_new,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
        results.append("Autosk Only with Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(time_budget/60)+'m'
        rs[1][2] = "Autofeat"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X_new.shape)
        res_df.append(rs[1])
    

    if mode == 0 or mode == 2:
        rs = run_as(X,y,target_ft, time_budget=time_budget+autofeat_time, include_preprocessors=None)   
        results.append("Autosk Only with Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(round((time_budget+autofeat_time)/60,2))+'m'
        rs[1][2] = "AutoSK"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X.shape)
        res_df.append(rs[1])
    if mode == 0 or mode == 3:
        rs = run_as(X,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
        results.append("Autosk Only without Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(round((time_budget+autofeat_time)/60,2))+'m'
        rs[1][2] = "None"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X.shape)
        res_df.append(rs[1])
    
    
    
    
    if mode == 0 or mode == 4:
        X_new = gen_feats_featools(X)
        rs = run_as(X_new,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
        results.append("Autosk with Featuretools: " + rs)

    if mode ==0 or mode == 5:
        rs = run_tpot(X,y,target_ft, time_budget=time_budget, include_preprocessors=None)   
        results.append("TPOT Only with Preprocessing: " + rs)
    
    
        
    print("===================================")
    print("Time budeget: ",time_budget)
    [print(x) for x in results]
    
    res_df =  pd.DataFrame(res_df, columns = ["Dataset","Time","Preprocessing","AutoML","Accuracy","F1","Shape","PipeLine"])
    res_df.drop(columns=["PipeLine"]).to_csv("results/"+df_path[5:])
    res_df.to_csv("results/pipe_"+df_path[5:])
    
    return res_df

In [37]:
cars = pd.read_csv("cars.csv")
trains = pd.read_csv("trains.csv")
#mol = pd.read_csv("mol.csv")

In [38]:
cars

,id,train_id,position,shape,len,sides,roof,wheels,load_shape,load_num
0,1,1,1,rectangle,short,not_double,none,2,circle,1
1,2,1,2,rectangle,long,not_double,none,3,hexagon,1
2,3,1,3,rectangle,short,not_double,peaked,2,triangle,1
3,4,1,4,rectangle,long,not_double,none,2,rectangle,3
4,5,2,1,rectangle,short,not_double,flat,2,circle,2
...,...,...,...,...,...,...,...,...,...,...
58,59,19,2,rectangle,long,not_double,flat,2,rectangle,3
59,60,19,3,rectangle,long,not_double,none,2,rectangle,0
60,61,19,4,u_shaped,short,not_double,none,2,triangle,1
61,62,20,1,rectangle,long,not_double,flat,3,hexagon,1


In [39]:
trains

,id,direction
0,1,east
1,2,east
2,3,east
3,4,east
4,5,east
5,6,east
6,7,east
7,8,east
8,9,east
9,10,east


In [40]:
df = pd.merge(cars, trains, left_on='train_id', right_on='id').drop(columns=["id_x"])

In [41]:
#df.to_csv("trains_merged.csv",index=False)

In [44]:
!rm -r tmp
df = "trains_merged.csv"
target_ft = "direction"
run_test(df, target_ft, mode=2 ,time_budget=30)

[WARNING] [2020-08-04 10:29:12,095:AutoML(1):36409f1e5278b605b6692aab76be8814] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.606464)
[WARNING] [2020-08-04 10:29:12,098:AutoML(1):36409f1e5278b605b6692aab76be8814] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-08-04 10:29:29,937:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.


Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7fec5e013518>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_directories(force=False)
  File "/usr/local/lib/python3.6/dist-packages/autosklearn/util/backend.py", line 136, in delete_directories
    % self.output_directory)
ValueError: Failed to delete output dir: ./tmp/autosklearn_regression_example_out because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.


Time budeget:  30
Autosk Only with Preprocessing: ['Accuracy score - 1.0', 'F1 score - 1.0']


FileNotFoundError: [Errno 2] No such file or directory: 'results/s_merged.csv'

In [48]:
entities = {
    "cars" : (cars, "id"),
    "trains" : (trains, "od",),
}

In [50]:
relationships = [("cars", "id", "trains", "id")]

In [51]:
trains

,id,direction
0,1,east
1,2,east
2,3,east
3,4,east
4,5,east
5,6,east
6,7,east
7,8,east
8,9,east
9,10,east


In [ ]:
feature_matrix_customers, features_defs = ft.dfs(entities=entities,
                                                  relationships=relationships,
   ....:                                                  target_entity="customers")